In [1]:
import requests
import os,json
from os.path import join, isdir
from pprint import pprint
import itertools as it
import dask
import re
from bs4 import BeautifulSoup
import string 
import numpy as np


In [2]:
class SoCInterface:
    def __init__(self):
        self.SoCPrefix = 'http://web-app.usc.edu/web/soc/api' 
    '''
    get the list of active instructors of the given semester. 
    Take ~10 secs to run 
    parameter: 
        semester, str, for instance: '20193' (2019 Fall)
    return:
        prof name, set, for instance: {(first, last), (first, last)...}
    '''
    def get_all_instructors(self, semester):
        url = '/'.join([self.SoCPrefix,'depts', semester]) # form a valid url
        schoolList = self.__make_request(url)['department']
        deptList = list(map(self.__dept_extractor, schoolList)) 
        deptList = list(it.chain.from_iterable(deptList)) # flatten the data structure, [{code:csci, name:comp science},{}]
        # now use get instructor info (multithreading)
        multithreader = [ dask.delayed(self.__get_dept_course)(dept,semester) for dept in deptList]
        # flattened result
        return list(set(list(it.chain.from_iterable(dask.compute(*multithreader, scheduler = 'threads',num_workers=12))) ))

    #---------------------------
    # private helpers 
    '''
    retrieve information from the given url 
    parameter:
        url: a valid url 
    return:
        a json file 
    '''
    def __make_request(self, url):
        try:
            response = requests.post(url)
        except: # in case of time-out, reconnect
            response = requests.post(url)
        return response.json()
    '''
    extract department names
    parameter: 
        dictionary: a dictionary of school information
    return:
        a list of departments under the given school, list of str
    '''
    def __dept_extractor(self, dictionary):
        try:
            if type(dictionary['department']) == list:
                return dictionary['department']
            else:
                return [dictionary['department']] # some school only has one department 
        except:
            return [dictionary] # some school might not have sub-department
    '''
    extract course information 
    parameter: 
        courseData: dictionary generated by __get_dept_course
    return:
        list of intructors of an given class, list of tuple
    '''
    def __course_extractor(self, courseData):
        if courseData['canceled'] == 'Y' or courseData['canceled'] == 'y':
            return []# return an empty dict if the session is cancled
        if 'instructor' in courseData:
            if type(courseData['instructor']) == dict: # mutiple instructors
                return [(courseData['instructor']['first_name'], courseData['instructor']['last_name']) ]
            else: # multiple prof for the same section
                return [(prof['first_name'], prof['last_name']) for prof in courseData['instructor']]
        else:
            return []
        
    '''
    get all courses under the same department
    parameter: 
        deptInfo, dictionary generated by getAllInstructors
        semester, str, for instance: '20193' (2019 Fall)
    return: a list of professors under the given department, a list of tuple
    '''
    def __get_dept_course(self,deptInfo, semester):
        url = '/'.join([self.SoCPrefix,'classes' ,deptInfo['code'], semester])
        courseList = self.__make_request(url)['OfferedCourses']['course']
        if type(courseList) == dict: # some dept has only one course
            courseList = [courseList]
        profList = []
        for course in courseList: 
            if type(course['CourseData']['SectionData']) == dict:  # some course has only one session
                profList += [self.__course_extractor(course['CourseData']['SectionData'])]
            else:
                profList += list(map(self.__course_extractor, course['CourseData']['SectionData']))
        return list(it.chain.from_iterable(profList)) # flatten the list

In [3]:
class RMPInterface:
    def __init__(self, outDir):
        if not isdir(outDir):
            os.mkdir (outDir)
        self.outDir = outDir
        self.RMPPrefix = 'https://www.ratemyprofessors.com'
        self. queryBody = '/search.jsp?queryoption=HEADER&queryBy=teacherName&schoolName=University+of+Southern+California&schoolID=1381&query='
    def update_prof_DB(self, profList):
        multithreader = [ dask.delayed(self.__get_prof_info)(prof) for prof in profList]
        profScoreList = list(dask.compute(*multithreader, scheduler = 'threads', num_workers=12))
        profScoreList = list (filter (lambda profInfo: len(profInfo)>0, profScoreList))
        
        #with open(join(self.outDir,'profInfoDB.json'), 'w') as outfile:
        #    json.dump(dict(profScoreList), outfile)
        return profScoreList
    '''    
    retrieve information about the given professor
    parameter:
        profName, tuple, (first name, last name)
    return: 
        information about the prof, tuple, for instance:
        (name, {department name 1: (score, # of raters), department name 2: (score, # of raters)}) 
        a name may have several departments because people may have the same name
    '''
    def __get_prof_info(self, profName):
        print ('processing prof:', profName)
        url = ''.join([self.RMPPrefix,self.queryBody,'+'.join(profName)])
        profListPage = BeautifulSoup(self.__make_request(url), 'html.parser')
        relatedProf = []
        for listings in  profListPage.find_all('li', {'class':'listing PROFESSOR'}):
            profUrl = self.RMPPrefix + listings.find('a', href=True)['href']# link to prof's individual page
            dept = listings.find('span', attrs = {'class':'sub'}).text.split(',')[-1]
            profInfoPage = BeautifulSoup(self.__make_request(profUrl), 'html.parser')
            rateTuple = self.__extract_prof_info(profInfoPage)
            if rateTuple != tuple(): # in case no information found 
                relatedProf.append (rateTuple)
        relatedProf = np.array(list (filter (lambda x:len(x)>1, relatedProf)))
        try:
            return (' '.join(profName), [np.mean(relatedProf[:,0]), np.mean(relatedProf[:,1]), sum(relatedProf[:,2])])
        except:
            return (' '.join(profName), [])
    '''
    make request and return text data
    parameter:
        url: a valid url pointing to RMP pages
    return:
        web content, str
    '''
    def __make_request(self, url):
        try:
            response = requests.post(url)
        except: # in case of time-out, reconnect
            response = requests.post(url)
        return response.text
    
    def __extract_prof_info(self, profInfoPage):
        try:
            grade = profInfoPage.find_all('div', {'class': 'grade'}, limit=2) # get avg. RMP score
            # get num of rater
            raterNum = re.findall(r'\b\d+\b',profInfoPage.find('div', {'class': 'table-toggle rating-count active'}).text)
            avgScore, diffLevel = float(grade[0].text),  float(re.findall(r'\b\d+\b',grade[1].text)[0])/100
            return [ avgScore, diffLevel, int(raterNum[0])]
        except:
            return []
        
    

In [5]:
%%time
gateway = SoCInterface()
profs = gateway.get_all_instructors('20193')


Wall time: 13 s


In [6]:
%%time
gateway = RMPInterface('.')
#gateway.update_prof_DB([('aaron', 'cote')])
profInfo = gateway.update_prof_DB(profs)

processing prof: ('Matthew', 'Wheeler')
processing prof: ('Diane', 'Badame')
processing prof: ('Hashem', 'Pesaran')
processing prof: ('David', 'Belson')
processing prof: ('Laurence', 'Gertz')
processing prof: ('Paul', 'Moorman')
processing prof: ('Tamar', 'Gregorian')
processing prof: ('Tiffany', 'Cole')
processing prof: ('Maria', 'Salnikova')
processing prof: ('Carly', 'Kenkel')
processing prof: ('Sara', 'Semborski')
processing prof: ('Allissa', 'Richardson')
processing prof: ('Ellen', 'Wayland-Smith')
processing prof: ('Dave', "O'Brien")
processing prof: ('Caroline', 'Cicero')
processing prof: ('Gian-Maria', 'Annovi')
processing prof: ('Jerry', 'Lucido')
processing prof: ('Arman', 'Nadershahi')
processing prof: ('Andrew', 'Guzman')
processing prof: ('Brandi', 'Wells')
processing prof: ('Larry', 'Livingston')
processing prof: ('Janet', 'Batchler')
processing prof: ('Artineh', 'Samkian')
processing prof: ('Ken', 'Alexander')
processing prof: ('Charlie', 'Sarosy')
processing prof: ('Jim

processing prof: ('Alberto', 'Vallejo')
processing prof: ('Caroline', 'Hu')
processing prof: ('Mike', 'Ananny')
processing prof: ('Sandra', 'Batista')
processing prof: ('Jesse', 'Vigil')
processing prof: ('Robert', 'Young')
processing prof: ('Shahin', 'Nazarian')
processing prof: ('Jesus', 'Dominguez')
processing prof: ('Joshua', 'Goldstein')
processing prof: ('Elahe', 'Nezami')
processing prof: ('David', "D'Argenio")
processing prof: ('Curtis', 'Okamoto')
processing prof: ('Anthony', 'Maddox')
processing prof: ('Daniel', 'Murphy')
processing prof: ('Aaron', 'Cote')
processing prof: ('Rachel', 'Gezerseh')
processing prof: ('Scott', 'Spencer')
processing prof: ('Douglas', 'Shook')
processing prof: ('Agatha', 'King')
processing prof: ('Naomi', 'Warren')
processing prof: ('Laura', 'Cardinal')
processing prof: ('Ahmad', 'Besaratinia')
processing prof: ('Stephen', 'Child')
processing prof: ('Shoshi', 'Kanokohata')
processing prof:processing prof: ('Velina', 'Hasu Houston')
 ('Trina', 'Hoefl

processing prof: ('Andrea', 'Armani')
processing prof: ('Quyen', 'Pham')
processing prof: ('Jessica', 'Kanoski')
processing prof: ('John', 'Brennan')
processing prof: ('Bill', 'Yahraus')
processing prof: ('Stacey', 'Carter')
processing prof: ('Ajita', 'Robinson')
processing prof: ('Kevin', 'Meza')
processing prof: ('Steve', 'Kao')
processing prof: ('Vitaly', 'Kresin')
processing prof: ('Nikhil', 'Bhatia')
processing prof: ('Thomas', 'McManus')
processing prof: ('Michael', 'Renov')
processing prof: ('Dawn', 'Porter')
processing prof: ('Ama', 'Saran')
processing prof: ('Paul', 'Lazarus')
processing prof: ('Hyungjin', 'Eoh')
processing prof: ('Scott', 'Wiltermuth')
processing prof: ('Danielle', 'Luchetta')
processing prof: ('Madhu', 'Thangavelu')
processing prof: ('Chevon', 'Hicks')
processing prof: ('Robert', 'Hanning')
processing prof: ('Tom', 'Jacobson')
processing prof: ('Diane', 'Comi')
processing prof: ('Wichada', 'La Motte-Kerr')
processing prof: ('Florence', 'Hofman')
processing p

processing prof: ('Kiranjeet', 'Dhillon')
processing prof: ('Brett', 'Gordon')
processing prof: ('Olu', 'Orange')
processing prof: ('Rosamaria', 'Alamo')
processing prof: ('Jose', 'Zavala')
processing prof: ('Smrity', 'Randhawa')
processing prof: ('Stephen', 'Jones')
processing prof: ('Hue', 'Wang')
processing prof: ('Marisa', 'Perdomo')
processing prof: ('David', 'Linton')
processing prof: ('Debra', 'Waters-Roman')
processing prof: ('Cecil', 'Jackson')
processing prof: ('Kimberly', 'West-Faulcon')
processing prof: ('Andrea', 'Hollingshead')
processing prof: ('Daniel', 'Schnur')
processing prof: ('Claire', 'Bono')
processing prof:processing prof: ('Kevin', 'Kane')
 ('Mimi', 'Gramatky Stradling')
processing prof: ('Eugenia', 'Weiss')
processing prof: ('Amy', 'Hodge')
processing prof: ('Rook', 'Campbell')
processing prof: ('Deniz', 'Rudin')
processing prof: ('Stefan', 'Pollack')
processing prof: ('Isabel', 'Wade')
processing prof: ('Jason', 'Fulman')
processing prof:processing prof:proce

processing prof: ('David', 'Measer')
processing prof: ('Christian', 'Phillips')
processing prof: ('Zivia', 'Wilson Sweeney')
processing prof: ('Eric', 'Coe')
processing prof: ('Sheila', 'Briggs')
processing prof: ('Khalil', 'Iskarous')
processing prof: ('John', 'Carpten')
processing prof: ('Jasmine', 'Bryant')
processing prof: ('Tyan', 'Parker Dominguez')
processing prof: ('Andrew', 'Justice')
processing prof: ('Louis', 'Goldstein')
processing prof: ('Timothy', 'Werlinger')
processing prof: ('James', 'McHugh')
processing prof: ('Dan', 'Shaner')
processing prof: ('Sean', 'Nye')
processing prof: ('Alison', "D'Amato")
processing prof: ('Deborah', 'Pitts')
processing prof: ('Charles', 'Gutierrez')
processing prof: ('Geoff', 'Boeing')
processing prof: ('Robert', 'Scheer')
processing prof: ('Stephanie', 'Bower')
processing prof:processing prof: ('Gayla', 'Margolin')
 ('Nestor', 'Perez Arancibia')
processing prof: ('John', 'De Mita')
processing prof: ('Pierson', 'Clair')
processing prof: ('Ja

...

processing prof: ('Rolf', 'Kestermann')
processing prof: ('John', 'Matsusaka')
processing prof: ('Sharon', 'Merle-Lieberman')
processing prof: ('David', 'Quinn')
processing prof: ('Paul', 'Hodgins')
processing prof: ('Nathan', 'Cherry')
processing prof: ('Ali', 'Zadeh')
processing prof: ('Patrick', 'Walsh')
processing prof: ('Leslie', 'King')
processing prof: ('Leah', 'Pate')
processing prof: ('Nathan', 'Perl-Rosenthal')
processing prof: ('Lauren', 'Matchison')
processing prof: ('John', 'Wilson')
processing prof: ('Alfred', 'Brown')
processing prof: ('Jessica', 'Parr')
processing prof: ('Martha', 'Townsend')
processing prof: ('Matthew', 'Manos')
processing prof: ('Steve', 'Vankanegan')
processing prof: ('Richard', 'Holland')
processing prof: ('Yunah', 'Chung')
processing prof: ('Lindsey', 'Munday')
processing prof: ('Jeff', 'Guh')
processing prof: ('Matthew', 'Lait')
processing prof: ('Bob', 'Bridges')
processing prof: ('David', 'Barnhart')
processing prof: ('Davina', 'Wolter')
process

processing prof: ('Nicholas', 'Lucas')
processing prof: ('Amy', 'Bresnahan')
processing prof: ('Mark', 'Moore')
processing prof: ('Ruth', 'Weisberg')
processing prof: ('John', 'Chang')
processing prof: ('Hye Jin', 'Lee')
processing prof: ('Martine', 'Culty')
processing prof: ('John', 'Hall')
processing prof: ('Andrew', 'Kutchera')
processing prof: ('Adam', 'Silverman')
processing prof: ('Richard', 'Antaramian')
processing prof: ('Meiqing', 'Zhang')
processing prof: ('Joan', 'Tardibuono')
processing prof: ('Mary', 'Hillier')
processing prof: ('Tyler', 'Patterson')
processing prof: ('Douglas', 'Becker')
processing prof: ('Margaret', 'Nelson')
processing prof: ('Nitin', 'Kale')
processing prof: ('Rodney', 'To')
processing prof: ('David', 'Bringhurst')
processing prof: ('Stanley', 'Augustine')
processing prof: ('Joseph', 'Cortez')
processing prof: ('Sean', 'Holt')
processing prof: ('Kairos', 'Llobrera')
processing prof: ('Phyllis', 'Pollack')
processing prof: ('Jason', 'Doctor')
processing

processing prof: ('Wenhao', 'Li')
processing prof: ('Robert', 'Turner')
processing prof: ('Ed', 'Reynolds')
processing prof: ('Alison', 'Hirsch')
processing prof: ('Gregory', 'Chesley')
processing prof: ('Paul', 'Donald')
processing prof: ('Mihailo', 'Jovanovic')
processing prof: ('John', 'Eklund')
processing prof: ('Amber', 'Richert')
processing prof: ('Shafiqa', 'Ahmadi')
processing prof: ('Sibyl', 'Wickersheimer')
processing prof: ('Bernadene', 'Blaha')
processing prof: ('Paula', 'Patnoe-Woodley')
processing prof: ('Jules', 'Kabat')
processing prof: ('Susan', 'Bain')
processing prof: ('Barnet', 'Kellman')
processing prof: ('Lodovico', 'Pizzati')
processing prof: ('Deborah', 'Glynn')
processing prof: processing prof: ('Jodie', 'Gates')
('Garry', 'Schyman')
processing prof: ('Michael', 'Hurlburt')
processing prof: ('John', 'Loper')
processing prof: ('James', 'Nokes')
processing prof: ('Maged', 'Dessouky')
processing prof: ('Hernan', 'Galperin')
processing prof: ('Omar', 'El Sawy')
pro

processing prof: ('Paul', 'Bader')
processing prof: ('B K Elizabeth', 'Kim')
processing prof: ('Christopher', 'Laine')
processing prof:processing prof:  ('Sanda', 'Win')
('Tien', 'Ng')
processing prof: ('Mike', 'Munoz')
processing prof: ('Melissa', 'Eckhause')
processing prof: ('Chiaoyun', 'Kuo')
processing prof: ('Parul', 'Bhatia')
processing prof: ('Robert', 'Lu')
processing prof: ('Roland', 'Wahlroos-Ritter')
processing prof: ('Steven', 'Fleming')
processing prof: ('Mark', 'Thompson')
processing prof: ('Sung-Hwa', 'Park')
processing prof: ('John', 'Monterosso')
processing prof: ('Behnam', 'Jafarpour')
processing prof: ('Darry', 'Sragow')
processing prof: ('John', 'Roach')processing prof: ('Frederic', 'Clark')

processing prof: ('Georgia', 'Jeffries')
processing prof: ('Rebecca', 'Lonergan')
processing prof: ('Suraiya', 'Rasheed')
processing prof: ('Jessica', 'Ho')
processing prof: ('Armand', 'Tanguay')
processing prof: ('Bryant', 'Adibe')
processing prof: ('Viola', 'Lasmana')
proces

processing prof: ('Adrian', 'Donato')
processing prof: ('Thomas', 'Mueller')
processing prof: ('Stanislav', 'Minsker')
processing prof: ('Gilberto', 'Gutierrez')
processing prof: ('Giorgia', 'Quadrato')
processing prof: ('Martin', 'Villares Ogaza')
processing prof: ('Susan', 'Kresin')
processing prof: ('Charles', 'Tang')
processing prof: ('Juli', 'Kirkpatrick')
processing prof: ('Kasia', 'Anderson')
processing prof: ('Michael', 'Madden')
processing prof: ('Carl', 'Castro')
processing prof: ('Rena', 'Heinrich')
processing prof: ('Caitlin', "O'Connell")
processing prof: ('David', 'Treuer')
processing prof: ('James', 'Deiner')
processing prof: ('Lucy V', 'Lee')
processing prof: ('Nicholas', 'Bredie')
processing prof: ('Arianna', 'Uhalde')
processing prof: ('Claudia', 'Kropf')
processing prof: ('Mark', 'Redekopp')
processing prof: ('Steven', 'Martin')
processing prof: ('Vanessa', 'Schwartz')
processing prof: ('Xavier', 'Fumat')
processing prof: ('Greg', 'Autry')
processing prof: ('Houda', 

In [7]:
        
with open(join('.','profInfoDB.json'), 'w') as outfile:
    json.dump(dict(profInfo), outfile)

In [8]:
dict(profInfo)

{'Kathleen Dowdey': [],
 'Ivan Nikkhoo': [4.2, 0.83, 6.0],
 'Andrew Goodney': [3.6, 0.65, 49.0],
 'James Diewald': [],
 'John Brekke': [],
 'Ratika Narag': [3.0, 0.55, 25.0],
 'Maria Mercedes Fages Agudo': [5.0, 1.0, 14.0],
 'Geoffrey Cowan': [4.0, 0.77, 16.0],
 'Shalini Gupta': [],
 'Lewis Rothbart': [],
 'Kathryn Dickason': [],
 'Kimberly Morris': [],
 'Eve Lee': [4.5, 0.75, 11.0],
 'Sarah Mojarad': [],
 'Ali Abbas': [],
 'Margaret Nelson': [],
 'Karen Anderson': [],
 'Elizabeth Misener': [4.6, 1.0, 13.0],
 'Laura Loyola': [],
 'Michael Hannan': [],
 'Thomas Cummings': [4.1, 1.0, 37.0],
 'Betsy Seder': [],
 'Cristy Lytal': [],
 'Su Jung Kim': [],
 'Martin Gundersen': [],
 'Mark Frey': [],
 'Goretti Prieto Botana': [],
 'Amy Hodge': [],
 'Ronald Manderscheid': [],
 'Clay Wang': [],
 'Juan Araque': [],
 'Robertas Gabrys': [2.7, 0.23, 46.0],
 'Richard Green': [],
 'Emily Schulze': [],
 'Carol Geffner': [],
 'Sergio Sanudo-Wilhelmy': [],
 'Robert Young': [],
 'Chander Burgos': [],
 'Tanv